In [ ]:
!pip install chainer

In [ ]:
import os
import sys
sys.path.insert(1, "/kaggle/input/qcanet3/QCANet")
import numpy as np
from skimage import io
import src.tools.qca_net

<h3> Everything below is ported from original QCANet Chainer-based code </h3>

<h1> NSN inference </h1>


In [ ]:
import os
import copy
import chainer
from chainer import cuda

from skimage import transform as tr
from src.lib.utils import mirror_extension_image
from src.lib.model import Model_L2, Model_L4
from os import listdir


patchsize = [128,128,128]
stride = [64,64,64]
resolution = [1.0,1.0,2.18]
scaling = True
ndim = 3

## create model 
class_weight = np.array([1, 1]).astype(np.float32)
nsn = Model_L2(class_weight=class_weight, n_class=2, init_channel=16,
                   kernel_size=3, pool_size=2, ap_factor=2, gpu=1)
chainer.serializers.load_npz("/kaggle/input/qcanet3/QCANet/models/learned_nsn.npz", nsn, strict=False)
cuda.get_device_from_id(0).use()  # Make a specified GPU current
nsn.to_gpu()  # Copy the SegmentNucleus model to the GPU
model = nsn
# ndn = Model_L4(class_weight=class_weight, n_class=2, init_channel=12,
#                    kernel_size=5, pool_size=2, ap_factor=2, gpu=1)
# chainer.serializers.load_npz("/kaggle/input/qcanet3/QCANet/models/learned_ndn.npz", ndn, strict=False)
# cuda.get_device_from_id(0).use()  # Make a specified GPU current
# ndn.to_gpu()  # Copy the SegmentNucleus model to the GPU
# model = ndn

segbase = 'SegmentationImages'
# if not (pt.exists(self.opbase + self.psep + segbase)):
#     os.mkdir(self.opbase + self.psep + segbase)

folder_dir = "/kaggle/input/dl-reprod/Images/test/Images/"

# for test_images in os.listdir(folder_dir):
image_path = "/kaggle/input/dl-reprod/Images/test/Images/Emb1_t351.tif"

image = io.imread(image_path)
im_size = image.shape
if ndim == 2:
    ip_size = (int(image.shape[0] * resolution[1]), int(image.shape[1] * resolution[0]))
    sh = [int(stride[0]/2), int(stride[1]/2)]
elif ndim == 3:
    ip_size = (int(image.shape[0] * resolution[2]), int(image.shape[1] * resolution[1]), int(image.shape[2] * resolution[0]))
    sh = [int(stride[0]/2), int(stride[1]/2), int(stride[2]/2)]


ip_size = (int(image.shape[0] * resolution[2]), int(image.shape[1] * resolution[1]), int(image.shape[2] * resolution[0]))
sh = [int(stride[0]/2), int(stride[1]/2), int(stride[2]/2)]

image = tr.resize(image, ip_size, order = 1, preserve_range = True)
im_size_ip = image.shape

#added in myself
if scaling:
    image = image.astype(np.float32)
    image = (image - image.min()) / (image.max() - image.min())

''' calculation for pad size'''
if np.min(patchsize) > np.max(np.array(im_size) + np.array(sh)*2):
    if ndim == 2:
        pad_size = [patchsize[0], patchsize[1]]
    elif ndim == 3:
        pad_size = [patchsize[0], patchsize[1], patchsize[2]]
else:
    pad_size = []
    for axis in range(len(im_size_ip)):
        if (im_size_ip[axis] + 2*sh[axis] - patchsize[axis]) % stride[axis] == 0:
            stride_num = int((im_size_ip[axis] + 2*sh[axis] - patchsize[axis]) / stride[axis])
        else:
            stride_num = int((im_size_ip[axis] + 2*sh[axis] - patchsize[axis]) / stride[axis]) + 1
        pad_size.append(int(stride[axis] * stride_num + patchsize[axis]))

pre_img = np.zeros(pad_size)

if ndim == 2:
    image = mirror_extension_image(image=image, ndim=ndim, length=int(np.max(patchsize)))[patchsize[0]-sh[0]:patchsize[0]-sh[0]+pad_size[0], patchsize[1]-sh[1]:patchsize[1]-sh[1]+pad_size[1]]
    for y in range(0, pad_size[0]-stride[0], stride[0]):
        for x in range(0, pad_size[1]-stride[1], stride[1]):
            x_patch = image[y:y+patchsize[0], x:x+patchsize[1]]
            x_patch = np.expand_dims(np.expand_dims(x_patch.astype(np.float32), axis=0), axis=0)
#             if self.gpu >= 0:
#                 x_patch = cuda.to_gpu(x_patch)
            s_output = model(x=x_patch, t=None, seg=True)
#             if self.gpu >= 0:
#                 s_output = cuda.to_cpu(s_output)
            pred = copy.deepcopy((0 < (s_output[0][1] - s_output[0][0])) * 255)
            # Add segmentation image
            pre_img[y:y+stride[0], x:x+stride[1]] += pred[sh[0]:-sh[0], sh[1]:-sh[1]]
    seg_img = (pre_img > 0) * 255
    seg_img = seg_img[:im_size_ip[0], :im_size_ip[1]]
elif ndim == 3:
    image = mirror_extension_image(image=image, ndim=ndim, length=int(np.max(patchsize)))[patchsize[0]-sh[0]:patchsize[0]-sh[0]+pad_size[0], patchsize[1]-sh[1]:patchsize[1]-sh[1]+pad_size[1], patchsize[2]-sh[2]:patchsize[2]-sh[2]+pad_size[2]]
    for z in range(0, pad_size[0]-stride[0], stride[0]):
        for y in range(0, pad_size[1]-stride[1], stride[1]):
            for x in range(0, pad_size[2]-stride[2], stride[2]):
                x_patch = image[z:z+patchsize[0], y:y+patchsize[1], x:x+patchsize[2]]
                x_patch = np.expand_dims(np.expand_dims(x_patch.astype(np.float32), axis=0), axis=0)
                x_patch = cuda.to_gpu(x_patch)
                s_output = model(x=x_patch, t=None, seg=True)
                s_output = cuda.to_cpu(s_output)
                pred = copy.deepcopy((0 < (s_output[0][1] - s_output[0][0])) * 255)
                # Add segmentation image
                pre_img[z:z+stride[0], y:y+stride[1], x:x+stride[2]] += pred[sh[0]:-sh[0], sh[1]:-sh[1], sh[2]:-sh[2]]
    seg_img = (pre_img > 0) * 255
    seg_img = seg_img[:im_size_ip[0], :im_size_ip[1], :im_size_ip[2]]
seg_img_nsn = (tr.resize(seg_img, im_size, order = 1, preserve_range = True) > 0) * 255
# filename = os.path.join(self.opbase, segbase, os.path.basename(image_path)[:os.path.basename(image_path).rfind('.')] + '.tif')
#filename = self.opbase + self.psep + segbase + self.psep + 'segimg_t{0:03d}.tif'.format(int(image_path[image_path.rfind('/')+1:image_path.rfind('.')]))
# filename = f"nsn/ndn_{test_images}"
# io.imsave(filename, seg_img.astype(np.uint8))
#     print(seg_img.astype(np.uint16))

<h1> NDN inference </h1>

In [ ]:
import os
import copy
import chainer
from chainer import cuda

from skimage import transform as tr
from src.lib.utils import mirror_extension_image
from src.lib.model import Model_L2, Model_L4
from os import listdir


patchsize = [128,128,128]
stride = [64,64,64]
resolution = [1.0,1.0,2.18]
scaling = True
ndim = 3

## create model 
ndn = Model_L4(class_weight=class_weight, n_class=2, init_channel=12,
                   kernel_size=5, pool_size=2, ap_factor=2, gpu=1)
chainer.serializers.load_npz("/kaggle/input/qcanet3/QCANet/models/learned_ndn.npz", ndn, strict=False)
cuda.get_device_from_id(0).use()  # Make a specified GPU current
ndn.to_gpu()  # Copy the SegmentNucleus model to the GPU
model = ndn

segbase = 'SegmentationImages'
# if not (pt.exists(self.opbase + self.psep + segbase)):
#     os.mkdir(self.opbase + self.psep + segbase)

folder_dir = "/kaggle/input/dl-reprod/Images/test/Images/"

# for test_images in os.listdir(folder_dir):
image_path = "/kaggle/input/dl-reprod/Images/test/Images/Emb1_t351.tif"

image = io.imread(image_path)
im_size = image.shape
if ndim == 2:
    ip_size = (int(image.shape[0] * resolution[1]), int(image.shape[1] * resolution[0]))
    sh = [int(stride[0]/2), int(stride[1]/2)]
elif ndim == 3:
    ip_size = (int(image.shape[0] * resolution[2]), int(image.shape[1] * resolution[1]), int(image.shape[2] * resolution[0]))
    sh = [int(stride[0]/2), int(stride[1]/2), int(stride[2]/2)]


ip_size = (int(image.shape[0] * resolution[2]), int(image.shape[1] * resolution[1]), int(image.shape[2] * resolution[0]))
sh = [int(stride[0]/2), int(stride[1]/2), int(stride[2]/2)]

image = tr.resize(image, ip_size, order = 1, preserve_range = True)
im_size_ip = image.shape

#added in myself
if scaling:
    image = image.astype(np.float32)
    image = (image - image.min()) / (image.max() - image.min())

''' calculation for pad size'''
if np.min(patchsize) > np.max(np.array(im_size) + np.array(sh)*2):
    if ndim == 2:
        pad_size = [patchsize[0], patchsize[1]]
    elif ndim == 3:
        pad_size = [patchsize[0], patchsize[1], patchsize[2]]
else:
    pad_size = []
    for axis in range(len(im_size_ip)):
        if (im_size_ip[axis] + 2*sh[axis] - patchsize[axis]) % stride[axis] == 0:
            stride_num = int((im_size_ip[axis] + 2*sh[axis] - patchsize[axis]) / stride[axis])
        else:
            stride_num = int((im_size_ip[axis] + 2*sh[axis] - patchsize[axis]) / stride[axis]) + 1
        pad_size.append(int(stride[axis] * stride_num + patchsize[axis]))

pre_img = np.zeros(pad_size)

if ndim == 2:
    image = mirror_extension_image(image=image, ndim=ndim, length=int(np.max(patchsize)))[patchsize[0]-sh[0]:patchsize[0]-sh[0]+pad_size[0], patchsize[1]-sh[1]:patchsize[1]-sh[1]+pad_size[1]]
    for y in range(0, pad_size[0]-stride[0], stride[0]):
        for x in range(0, pad_size[1]-stride[1], stride[1]):
            x_patch = image[y:y+patchsize[0], x:x+patchsize[1]]
            x_patch = np.expand_dims(np.expand_dims(x_patch.astype(np.float32), axis=0), axis=0)
#             if self.gpu >= 0:
#                 x_patch = cuda.to_gpu(x_patch)
            s_output = model(x=x_patch, t=None, seg=True)
#             if self.gpu >= 0:
#                 s_output = cuda.to_cpu(s_output)
            pred = copy.deepcopy((0 < (s_output[0][1] - s_output[0][0])) * 255)
            # Add segmentation image
            pre_img[y:y+stride[0], x:x+stride[1]] += pred[sh[0]:-sh[0], sh[1]:-sh[1]]
    seg_img = (pre_img > 0) * 255
    seg_img = seg_img[:im_size_ip[0], :im_size_ip[1]]
elif ndim == 3:
    image = mirror_extension_image(image=image, ndim=ndim, length=int(np.max(patchsize)))[patchsize[0]-sh[0]:patchsize[0]-sh[0]+pad_size[0], patchsize[1]-sh[1]:patchsize[1]-sh[1]+pad_size[1], patchsize[2]-sh[2]:patchsize[2]-sh[2]+pad_size[2]]
    for z in range(0, pad_size[0]-stride[0], stride[0]):
        for y in range(0, pad_size[1]-stride[1], stride[1]):
            for x in range(0, pad_size[2]-stride[2], stride[2]):
                x_patch = image[z:z+patchsize[0], y:y+patchsize[1], x:x+patchsize[2]]
                x_patch = np.expand_dims(np.expand_dims(x_patch.astype(np.float32), axis=0), axis=0)
                x_patch = cuda.to_gpu(x_patch)
                s_output = model(x=x_patch, t=None, seg=True)
                s_output = cuda.to_cpu(s_output)
                pred = copy.deepcopy((0 < (s_output[0][1] - s_output[0][0])) * 255)
                # Add segmentation image
                pre_img[z:z+stride[0], y:y+stride[1], x:x+stride[2]] += pred[sh[0]:-sh[0], sh[1]:-sh[1], sh[2]:-sh[2]]
    seg_img = (pre_img > 0) * 255
    seg_img = seg_img[:im_size_ip[0], :im_size_ip[1], :im_size_ip[2]]
seg_img_nsn = (tr.resize(seg_img, im_size, order = 1, preserve_range = True) > 0) * 255
# filename = os.path.join(self.opbase, segbase, os.path.basename(image_path)[:os.path.basename(image_path).rfind('.')] + '.tif')
#filename = self.opbase + self.psep + segbase + self.psep + 'segimg_t{0:03d}.tif'.format(int(image_path[image_path.rfind('/')+1:image_path.rfind('.')]))
# filename = f"nsn/ndn_{test_images}"
# io.imsave(filename, seg_img.astype(np.uint8))
#     print(seg_img.astype(np.uint16))

In [ ]:
!zip -r /kaggle/working/zips/ndn.zip /kaggle/working/ndn -i \*.tif

<h1> Watershed post-processing </h1>

In [ ]:
from scipy import ndimage
from skimage.segmentation import watershed

seg_img = seg_img_nsn

### Detection Phase ###
det_img = seg_img_ndn

### Post-Processing ###
if det_img.sum() > 0:
    distance = ndimage.distance_transform_edt(seg_img)
    wsimage = watershed(-distance, det_img, mask=seg_img)
else:
    wsimage = morphology.label(seg_img, neighbors=4)
labels = np.unique(wsimage)
wsimage = np.searchsorted(labels, wsimage)
# filename = os.path.join(opbase, wsbase, os.path.basename(image_path)[:os.path.basename(image_path).rfind('.')] + '.tif')
# filename = opbase + psep + wsbase + psep + 'ws_t{0:03d}.tif'.format(int(image_path[image_path.rfind('/')+1:image_path.rfind('.')]))
io.imsave("wsimage_try.tif", wsimage.astype(np.uint16))